In [ ]:
import nasdaqdatalink
import numpy as np
import os
import pandas as pd

from IPython.display import display
from load_api_keys import load_api_keys
from pathlib import Path
from settings import config

# Load API keys from the environment
api_keys = load_api_keys()

# Get the environment variable for where data is stored
DATA_DIR = config("DATA_DIR")

def ndl_pull_data(
    base_directory,
    ticker: str,
    source: str,
    asset_class: str,
    excel_export: bool,
    pickle_export: bool,
    output_confirmation: bool,
) -> pd.DataFrame:
    
    """
    Download daily price ata from Nasdaq Data Link and add many missing columns and export it.

    Parameters:
    -----------
    base_directory
        Root path to store downloaded data.
    ticker : str
        Ticker symbol to download.
    source : str
        Name of the data source (e.g., 'Nasdaq_Data_Link').
    asset_class : str
        Asset class name (e.g., 'Equities').
    excel_export : bool
        If True, export data to Excel format.
    pickle_export : bool
        If True, export data to Pickle format.
    output_confirmation : bool
        If True, print confirmation message.

    Returns:
    --------
    df : pd.DataFrame
        DataFrame containing the downloaded data.
    """

    # Command to pull data
    # If start date and end date are not specified the entire data set is included
    df = nasdaqdatalink.get_table("QUOTEMEDIA/PRICES", ticker=ticker, paginate=True, api_key=api_keys["NASDAQ_DATA_LINK_KEY"])

    # Sort columns by date ascending
    df.sort_values('date', ascending = True, inplace = True)

    # Rename the date column
    df.rename(columns = {'date':'Date'}, inplace = True)

    # Set index to date column
    df.set_index('Date', inplace = True)

    # Replace all split values of 1.0 with NaN
    df['split'] = df['split'].replace(1.0, np.nan)

    # Create a new data frame with split values only
    df_splits = df.drop(columns = {'ticker', 'open', 'high', 'low', 
                                   'close', 'volume', 'dividend', 
                                   'adj_open', 'adj_high', 
                                   'adj_low', 'adj_close', 
                                   'adj_volume'}).dropna()

    # Create a new column for cumulative split
    df_splits['Cum_Split'] = df_splits['split'].cumprod()

    # Drop original split column before combining dataframes
    df_splits.drop(columns = {'split'}, inplace = True)

    # Merge df and df_split dataframes
    df_comp = pd.merge(df, df_splits, on='Date', how="outer")

    # Forward fill for all cumulative split values
    df_comp['Cum_Split'] = df_comp['Cum_Split'].ffill()

    # Replace all split and cumulative split values of NaN with 1.0 to have complete split values
    df_comp['split'] = df_comp['split'].replace(np.nan, 1.0)
    df_comp['Cum_Split'] = df_comp['Cum_Split'].replace(np.nan, 1.0)

    # Calculate the non adjusted prices based on the splits only
    df_comp['non_adj_open_split_only'] = df_comp['open'] * df_comp['Cum_Split']
    df_comp['non_adj_high_split_only'] = df_comp['high'] * df_comp['Cum_Split']
    df_comp['non_adj_low_split_only'] = df_comp['low'] * df_comp['Cum_Split']
    df_comp['non_adj_close_split_only'] = df_comp['close'] * df_comp['Cum_Split']
    df_comp['non_adj_dividend_split_only'] = df_comp['dividend'] * df_comp['Cum_Split']

    # Calculate the adjusted prices based on the splits
    df_comp['Open'] = df_comp['non_adj_open_split_only'] / df_comp['Cum_Split'][-1]
    df_comp['High'] = df_comp['non_adj_high_split_only'] / df_comp['Cum_Split'][-1]
    df_comp['Low'] = df_comp['non_adj_low_split_only'] / df_comp['Cum_Split'][-1]
    df_comp['Close'] = df_comp['non_adj_close_split_only'] / df_comp['Cum_Split'][-1]
    df_comp['Dividend'] = df_comp['non_adj_dividend_split_only'] / df_comp['Cum_Split'][-1]
    df_comp['Dividend_Pct_Orig'] = df_comp['dividend'] / df_comp['close']
    df_comp['Dividend_Pct_Adj'] = df_comp['Dividend'] / df_comp['Close']

    # Create directory
    directory = f"{base_directory}/{source}/{asset_class}/Daily"
    os.makedirs(directory, exist_ok=True)

    # Export to excel
    if excel_export == True:
        df_comp.to_excel(f"{directory}/{ticker}.xlsx", sheet_name="data")
    else:
        pass

    # Export to pickle
    if pickle_export == True:
        df_comp.to_pickle(f"{directory}/{ticker}.pkl")
    else:
        pass

    # Output confirmation
    if output_confirmation == True:
        print(f"The first and last date of data for {ticker} is: ")
        display(df_comp[:1])
        display(df_comp[-1:])
        print(f"NDL data complete for {ticker}")
        print(f"--------------------")
    else:
        pass

    return df_comp

ndl_pull_data(
    base_directory=DATA_DIR,
    ticker="TLT",
    source="Nasdaq_Data_Link",
    asset_class="Exchange_Traded_Funds",
    excel_export=True,
    pickle_export=True,
    output_confirmation=True,
)

In [5]:
import nasdaqdatalink
import numpy as np
import os
import pandas as pd

from IPython.display import display
from load_api_keys import load_api_keys
from pathlib import Path
from settings import config

# Load API keys from the environment
api_keys = load_api_keys()

# Get the environment variable for where data is stored
DATA_DIR = config("DATA_DIR")

def ndl_pull_data(
    base_directory,
    ticker: str,
    source: str,
    asset_class: str,
    excel_export: bool,
    pickle_export: bool,
    output_confirmation: bool,
) -> pd.DataFrame:
    
    """
    Download daily price ata from Nasdaq Data Link and add many missing columns and export it.

    Parameters:
    -----------
    base_directory
        Root path to store downloaded data.
    ticker : str
        Ticker symbol to download.
    source : str
        Name of the data source (e.g., 'Nasdaq_Data_Link').
    asset_class : str
        Asset class name (e.g., 'Equities').
    excel_export : bool
        If True, export data to Excel format.
    pickle_export : bool
        If True, export data to Pickle format.
    output_confirmation : bool
        If True, print confirmation message.

    Returns:
    --------
    df : pd.DataFrame
        DataFrame containing the downloaded data.
    """

    # Command to pull data
    # If start date and end date are not specified the entire data set is included
    df = nasdaqdatalink.get_table("NDAQ/USEDHADJ", symbol=ticker, paginate=True, api_key=api_keys["NASDAQ_DATA_LINK_KEY"])

    # Sort columns by date ascending
    df.sort_values('date', ascending = True, inplace = True)

    # Rename the date column
    df.rename(columns = {'date':'Date'}, inplace = True)

    # Set index to date column
    df.set_index('Date', inplace = True)

    # # Replace all split values of 1.0 with NaN
    # df['split'] = df['split'].replace(1.0, np.nan)

    # # Create a new data frame with split values only
    # df_splits = df.drop(columns = {'ticker', 'open', 'high', 'low', 
    #                                'close', 'volume', 'dividend', 
    #                                'adj_open', 'adj_high', 
    #                                'adj_low', 'adj_close', 
    #                                'adj_volume'}).dropna()

    # # Create a new column for cumulative split
    # df_splits['Cum_Split'] = df_splits['split'].cumprod()

    # # Drop original split column before combining dataframes
    # df_splits.drop(columns = {'split'}, inplace = True)

    # # Merge df and df_split dataframes
    # df_comp = pd.merge(df, df_splits, on='Date', how="outer")

    # # Forward fill for all cumulative split values
    # df_comp['Cum_Split'] = df_comp['Cum_Split'].ffill()

    # # Replace all split and cumulative split values of NaN with 1.0 to have complete split values
    # df_comp['split'] = df_comp['split'].replace(np.nan, 1.0)
    # df_comp['Cum_Split'] = df_comp['Cum_Split'].replace(np.nan, 1.0)

    # # Calculate the non adjusted prices based on the splits only
    # df_comp['non_adj_open_split_only'] = df_comp['open'] * df_comp['Cum_Split']
    # df_comp['non_adj_high_split_only'] = df_comp['high'] * df_comp['Cum_Split']
    # df_comp['non_adj_low_split_only'] = df_comp['low'] * df_comp['Cum_Split']
    # df_comp['non_adj_close_split_only'] = df_comp['close'] * df_comp['Cum_Split']
    # df_comp['non_adj_dividend_split_only'] = df_comp['dividend'] * df_comp['Cum_Split']

    # # Calculate the adjusted prices based on the splits
    # df_comp['Open'] = df_comp['non_adj_open_split_only'] / df_comp['Cum_Split'][-1]
    # df_comp['High'] = df_comp['non_adj_high_split_only'] / df_comp['Cum_Split'][-1]
    # df_comp['Low'] = df_comp['non_adj_low_split_only'] / df_comp['Cum_Split'][-1]
    # df_comp['Close'] = df_comp['non_adj_close_split_only'] / df_comp['Cum_Split'][-1]
    # df_comp['Dividend'] = df_comp['non_adj_dividend_split_only'] / df_comp['Cum_Split'][-1]
    # df_comp['Dividend_Pct_Orig'] = df_comp['dividend'] / df_comp['close']
    # df_comp['Dividend_Pct_Adj'] = df_comp['Dividend'] / df_comp['Close']

    # Create directory
    directory = f"{base_directory}/{source}/{asset_class}/Daily"
    os.makedirs(directory, exist_ok=True)

    # Export to excel
    if excel_export == True:
        # df_comp.to_excel(f"{directory}/{ticker}.xlsx", sheet_name="data")
        df.to_excel(f"{directory}/{ticker}.xlsx", sheet_name="data")
    else:
        pass

    # Export to pickle
    if pickle_export == True:
        # df_comp.to_pickle(f"{directory}/{ticker}.pkl")
        df.to_pickle(f"{directory}/{ticker}.pkl")
    else:
        pass

    # Output confirmation
    if output_confirmation == True:
        print(f"The first and last date of data for {ticker} is: ")
        display(df[:1])
        display(df[-1:])
        print(f"NDL data complete for {ticker}")
        print(f"--------------------")
    else:
        pass

    # return df_comp
    return df

ndl_pull_data(
    base_directory=DATA_DIR,
    ticker="TLT",
    source="Nasdaq_Data_Link",
    asset_class="Exchange_Traded_Funds",
    excel_export=True,
    pickle_export=True,
    output_confirmation=True,
)

The first and last date of data for TLT is: 


,symbol,symbol_new,composite_figi,share_class_figi,volume,low,high,open,close,volume_adj,low_adj,high_adj,open_adj,close_adj,action,value
Date,,,,,,,,,,,,,,,,
2014-01-02,TLT,TLT,BBG000BJKYW3,BBG001S8MLN3,8580808.0,101.69,102.39,101.72,102.17,8580808.0,74.062,74.572,74.084,74.412,None,None


,symbol,symbol_new,composite_figi,share_class_figi,volume,low,high,open,close,volume_adj,low_adj,high_adj,open_adj,close_adj,action,value
Date,,,,,,,,,,,,,,,,
2025-11-26,TLT,TLT,BBG000BJKYW3,BBG001S8MLN3,34098962.0,89.83,90.64,90.15,90.64,34098962.0,89.83,90.64,90.15,90.64,None,None


NDL data complete for TLT
--------------------


,symbol,symbol_new,composite_figi,share_class_figi,volume,low,high,open,close,volume_adj,low_adj,high_adj,open_adj,close_adj,action,value
Date,,,,,,,,,,,,,,,,
2014-01-02,TLT,TLT,BBG000BJKYW3,BBG001S8MLN3,8580808.0,101.6900,102.3900,101.72,102.17,8580808.0,74.0620,74.5720,74.084,74.412,None,None
2014-01-03,TLT,TLT,BBG000BJKYW3,BBG001S8MLN3,4083831.0,101.7600,102.4500,101.81,102.17,4083831.0,74.1130,74.6160,74.150,74.412,None,None
2014-01-06,TLT,TLT,BBG000BJKYW3,BBG001S8MLN3,7796228.0,102.3700,103.0000,102.38,102.60,7796228.0,74.5580,75.0170,74.565,74.725,None,None
2014-01-07,TLT,TLT,BBG000BJKYW3,BBG001S8MLN3,4428082.0,102.5600,102.9900,102.82,102.86,4428082.0,74.6960,75.0090,74.885,74.915,None,None
2014-01-08,TLT,TLT,BBG000BJKYW3,BBG001S8MLN3,8512987.0,102.1000,102.6900,102.45,102.58,8512987.0,74.3610,74.7910,74.616,74.711,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-20,TLT,TLT,BBG000BJKYW3,BBG001S8MLN3,32945001.0,88.9850,89.4200,88.99,89.23,32945001.0,88.9850,89.4200,88.990,89.230,None,None
2025-11-21,TLT,TLT,BBG000BJKYW3,BBG001S8MLN3,41938628.0,89.1950,89.6700,89.65,89.50,41938628.0,89.1950,89.6700,89.650,89.500,None,None
2025-11-24,TLT,TLT,BBG000BJKYW3,BBG001S8MLN3,30865695.0,89.7801,90.0665,89.95,90.01,30865695.0,89.7801,90.0665,89.950,90.010,None,None
